# Cifar-10 이미지 분류
### 전체 데이터 사용
### Data Augmentation 적용
### 출처: [Achieving 90% accuracy in Object Recognition ](https://appliedmachinelearning.blog/2018/03/24/achieving-90-accuracy-in-object-recognition-task-on-cifar-10-dataset-with-keras-convolutional-neural-networks/)

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.regularizers import l2
import numpy as np

#### 자료형 변환 및 스케일링
- X: 실수형으로 정규화
- Y: 1-hot encoding
    * airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

In [2]:
(X_train, y_train0), (X_test, y_test0) = cifar10.load_data()
print(X_train.shape, X_train.dtype)
print(y_train0.shape, y_train0.dtype)
print(X_test.shape, X_test.dtype)
print(y_test0.shape, y_test0.dtype)

(50000, 32, 32, 3) uint8
(50000, 1) uint8
(10000, 32, 32, 3) uint8
(10000, 1) uint8


In [3]:
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0

print(X_train.shape, X_train.dtype)

(50000, 32, 32, 3) float32


In [4]:
Y_train = tf.keras.utils.to_categorical(y_train0, 10)
Y_test = tf.keras.utils.to_categorical(y_test0, 10)
Y_train[:4]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

#### 모형 구현

In [5]:
np.random.seed(0)
weight_decay = 1e-4

In [6]:
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay), 
                 input_shape=X_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0

In [7]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(lr=0.001, decay=weight_decay), 
              metrics=['accuracy'])

In [8]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    if epoch > 40:
        lrate = 0.0003
    return lrate

In [9]:
#data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(X_train)

In [10]:
## to save checkpoint to use later
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

modelpath = "model/Cifar-10-best.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

In [11]:
from tensorflow.keras.callbacks import LearningRateScheduler

history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=64),
                    steps_per_epoch=X_train.shape[0] // 64, epochs=300,
                    verbose=1, validation_data=(X_test, Y_test),
                    callbacks=[LearningRateScheduler(lr_schedule),checkpointer,early_stopping_callback])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/300
780/781 [============================>.] - ETA: 0s - loss: 1.8842 - accuracy: 0.4265
Epoch 00001: val_loss improved from inf to 1.39975, saving model to model/Cifar-10-best.hdf5
781/781 [==============================] - 35s 44ms/step - loss: 1.8840 - accuracy: 0.4266 - val_loss: 1.3998 - val_accuracy: 0.5665 - lr: 0.0010
Epoch 2/300
781/781 [==============================] - ETA: 0s - loss: 1.2502 - accuracy: 0.5913
Epoch 00002: val_loss improved from 1.39975 to 1.29002, saving model to model/Cifar-10-best.hdf5
781/781 [==============================] - 34s 44ms/step - loss: 1.2502 - accuracy: 0.5913 - val_loss: 1.2900 - val_accuracy: 0.6069 - lr: 0.0010
Epoch 3/300
781/781 [==============================] - ETA: 0s - loss: 1.0610 - accuracy: 0.6566
Epoch 00003: val_loss improved from 1.29002 to 1.08945, saving model to model/Cifar-10-best.hdf5
781/781 [==============================] - 34s 44ms/st

Epoch 26/300
780/781 [============================>.] - ETA: 0s - loss: 0.5469 - accuracy: 0.8424
Epoch 00026: val_loss did not improve from 0.51279
781/781 [==============================] - 34s 44ms/step - loss: 0.5472 - accuracy: 0.8423 - val_loss: 0.6205 - val_accuracy: 0.8247 - lr: 0.0010
Epoch 27/300
780/781 [============================>.] - ETA: 0s - loss: 0.5491 - accuracy: 0.8410
Epoch 00027: val_loss did not improve from 0.51279
781/781 [==============================] - 34s 44ms/step - loss: 0.5492 - accuracy: 0.8409 - val_loss: 0.5445 - val_accuracy: 0.8508 - lr: 0.0010
Epoch 28/300
780/781 [============================>.] - ETA: 0s - loss: 0.5305 - accuracy: 0.8464
Epoch 00028: val_loss did not improve from 0.51279
781/781 [==============================] - 34s 44ms/step - loss: 0.5306 - accuracy: 0.8463 - val_loss: 0.5249 - val_accuracy: 0.8541 - lr: 0.0010
Epoch 29/300
781/781 [==============================] - ETA: 0s - loss: 0.5325 - accuracy: 0.8467
Epoch 00029: val_

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model
del model
model = load_model('model/Cifar-10-best.hdf5')

#### 모델 평가하기

In [ ]:
scores = model.evaluate(X_test, Y_test, batch_size=128, verbose=2)
print('\nAccuracy: %.4f' % scores[1])